# Somme de deux variables gaussiennes

- Create the `Normal` distribution $X$ with parameters $\mu=-5,\sigma=3$
- Create the `Normal` distribution $Y$ with parameters $\mu=5,\sigma=3$
- Create the multivariate `Distribution` with marginals $X$ and $Y$ and independent copula
- Create the `RandomVector` input random vector `inRV` defined with the distribution
- Define the symbolic function `f` such that 
$$
f(x,y)=x+y
$$
- Create the `RandomVector` output random vector `outRV` as composition of `f` and `inRV`
- Create three sample with sizes `N=100,1000,10000` of the output random vector `outRV`.
- For each sample, compute :

  - The mean,
  - The standard deviation,
  - The quantiles of level $\alpha$=`0.05,0.50,0.95`. 

- Now instead of the `Function` and random vectors, use the `+` operator to define the variable `Z=X+Y`.

- From this distribution `Z`, compute :
  - The mean,
  - The standard deviation,
  - The quantiles of level $\alpha$=`0.05,0.50,0.95`. 
- Compare the two results: which one is more accurate?

In [1]:
import math
import openturns as ot

# 1) 定义边缘分布 X, Y
X = ot.Normal(-5.0, 3.0)
Y = ot.Normal( 5.0, 3.0)

# 2) 二维独立联合分布
distXY = ot.ComposedDistribution([X, Y], ot.IndependentCopula(2))

# 3) 输入随机向量
inRV = ot.RandomVector(distXY)

# 4) 符号函数 f(x,y)=x+y
f = ot.SymbolicFunction(["x","y"], ["x + y"])

# 5) 输出随机向量 outRV = f(inRV)
outRV = ot.CompositeRandomVector(f, inRV)

# 6) 三个样本规模
Ns = [100, 1000, 10000]
alphas = [0.05, 0.50, 0.95]

def summarize_sample(rv, N, alphas):
    sample = rv.getSample(N)
    mean = sample.computeMean()                   # Point(1)
    sd   = sample.computeStandardDeviation()      # Point(1)
    qs   = [sample.computeQuantilePerComponent(a)[0] for a in alphas]
    return float(mean[0]), float(sd[0]), qs

print("=== Monte Carlo on outRV (f(X,Y)=X+Y) ===")
for N in Ns:
    m, s, qs = summarize_sample(outRV, N, alphas)
    print(f"N={N:5d} | mean={m:.4f} | sd={s:.4f} | quantiles {alphas} = {[round(q,4) for q in qs]}")

# 7) 用 “+” 定义 Z = X + Y （分布层面：两独立正态之和仍为正态）
#    解析结果：mu_Z = -5 + 5 = 0
#            sigma_Z = sqrt(3^2 + 3^2) = sqrt(18)
muZ = X.getMean()[0] + Y.getMean()[0]
sdZ = math.sqrt(X.getStandardDeviation()[0]**2 + Y.getStandardDeviation()[0]**2)
Z  = ot.Normal(muZ, sdZ)   # 直接得到 Z 的分布

# 8) 从分布 Z 计算均值、标准差、分位数
mZ  = Z.getMean()[0]
sZ  = Z.getStandardDeviation()[0]
qZs = [Z.computeQuantile(a)[0] for a in alphas]

print("\n=== Analytical distribution Z = X + Y (Normal) ===")
print(f"mean={mZ:.4f} | sd={sZ:.4f} | quantiles {alphas} = {[round(q,4) for q in qZs]}")


=== Monte Carlo on outRV (f(X,Y)=X+Y) ===
N=  100 | mean=-0.2378 | sd=4.3994 | quantiles [0.05, 0.5, 0.95] = [-7.3663, -0.3166, 7.4636]
N= 1000 | mean=-0.1093 | sd=4.2336 | quantiles [0.05, 0.5, 0.95] = [-7.2289, -0.1775, 7.0372]
N=10000 | mean=-0.0145 | sd=4.2362 | quantiles [0.05, 0.5, 0.95] = [-6.8414, -0.0555, 6.9169]

=== Analytical distribution Z = X + Y (Normal) ===
mean=0.0000 | sd=4.2426 | quantiles [0.05, 0.5, 0.95] = [-6.9785, 0.0, 6.9785]
